In [19]:
class BondOut:
    def __init__(self, kind, source):
        self.kind = kind      # what am I?
        self.source = source  # who produced me?
        
    def __str__(self):
        return f'{self.kind} from {self.source.name}'
    
class BondIn:
    def __init__(self, kind, receiver):
        self.kind = kind
        self.recip = receiver
        
    
    def getListApplicableSources(self, outputs):
        self.maybe = []
        for b in outputs:
            if b.source is self.recip:
                continue  # no taking own output
            
            if b.kind == self.kind:
                self.maybe.append(b)
                    
        return self.maybe
                    
    
    def __str__(self):
        return f'need a {self.kind} for {self.recip.name}'

class Token:
    def __init__(self, name, incoming, outgoing):
        globals()[name] = self
        self.name = name
        self.incoming = [BondIn(i, self) for i in incoming]
        self.outgoing = [BondOut(o, self) for o in outgoing]

        

Token('parameters', [], ['list.number'])
Token('summ', ['list.number'], ['number'])
Token('length', ['list.number'], ['number'])
Token('divide', ['number', 'number'], ['number'])
Token('ret', ['number'], [])


class Ribosome:
    def __init__(self, tokens):
        self.tokens = tokens
        
        self.inputs = [i for t in self.tokens for i in t.incoming]
        self.outputs = [i for t in self.tokens for i in t.outgoing]
        
        for b in self.inputs:
            b.getListApplicableSources(self.outputs)
            print(b)
            for m in b.maybe:
                print(' ', m)
            print()
            
        for b in self.inputs:
            
            
        
        
        
        
Ribosome([parameters, summ, length, divide, ret])

need a list.number for summ
  list.number from parameters

need a list.number for length
  list.number from parameters

need a number for divide
  number from summ
  number from length

need a number for divide
  number from summ
  number from length

need a number for ret
  number from summ
  number from length
  number from divide



for each possible input
get list of applicable outputs

In [20]:
b = Bond('foo', 3)
b.show()

AttributeError: 'Bond' object has no attribute 'show'

bond - type and requirements

eg. gives list.number "used 1+ times"
    "any number of inputs may take my resultant list"
    
eg. needs number "used 1 time"
    "I need exactly one numeric output" 
    
eg. needs bool "used 0-1 times"
    "I can have a bool provided, otherwise I default to <False>"
    


In [46]:
import numpy as np
from itertools import product

class Bond:
    def __init__(self, atom, direction, kind):
        self.kind = kind
        self.atom = atom
        
        assert direction in ('gives', 'takes')
        self.gives = direction == 'gives'
        self.takes = direction == 'takes'
        
bonds = [
    Bond('div', 'takes', 'number'),  # dividend
    Bond('div', 'takes', 'number'), # divisor
    Bond('div', 'gives', 'number'), # quotient
    
    Bond('len', 'takes', 'array'),
    Bond('len', 'gives', 'number'),
    
    Bond('sum', 'takes', 'array'),
    Bond('sum', 'gives', 'number'),
    
    Bond('func', 'gives', 'array'),  # param
    Bond('func', 'takes', 'number'), # output
]

inputs = [b for b in bonds if b.takes and b.kind=='number']
outputs = [b for b in bonds if b.gives and b.kind=='number']

class SolvingMatrix:
    def __init__(self, inputs, outputs):
        # cell status, either 
        #  -- -1 / no match
        #  --  1 / match
        #  --  0 / ???
        
        self.inputs = inputs
        self.outputs = outputs
        self.matrix = np.zeros((len(inputs), len(outputs)))
        
        for (i_idx, i), (o_idx, o) in product(
            enumerate(inputs), enumerate(outputs)):
            
            if o.atom == i.atom:
                self.matrix[i_idx][o_idx] = -1
                
        self.show()
        
    def show(self):
        print('  ', ', '.join(o.atom for o in self.outputs))
        for i, line in zip(self.inputs, self.matrix):
            print(i.atom, line)

SolvingMatrix(inputs, outputs)


   div, len, sum
div [-1.  0.  0.]
div [-1.  0.  0.]
func [0. 0. 0.]


In [23]:
1 in range(1,2)

True

In [ ]:
class Bond:
    def __init__(self, kind, param=False, result=False, 
                 optional=False):
        
        self.kind = kind   # int / str / etc
        
        assert param ^ result
        self.param = param   # an incoming bond / parameter
        self.result = result   # an outgoing bond / return value
        
        self.min_uses = 1
        if optional:
            self.min_uses = 0
            
        self.max_uses = 1
        if self.result:
            self.max_uses = 999
            # can be consumed by many other funcs
        


False

In [63]:
'''
ok = mod 3 or 5 equals 0
eg. 1..10 where ok -> [3, 5, 6, 9, 10]

sum 1..1000 where ok
'''

'''
stringSplosion = concat slices to 1..
eg. stringSplosion "Code" -> "CCoCodCode"
'''

'''
average = sum / length
eg. average [1, 2, 3] -> 2
    average [4, 5] -> 4.5

'''


'\naverage = sum / length\neg. average [1, 2, 3] -> 2\n    average [4, 5] -> 4.5\n\n'

In [65]:
    
class Bond:
    def __init__(self, kind, param=False, result=False, 
                 optional=False):
        
        self.kind = kind   # int / str / etc
        
        assert param ^ result
        self.param = param   # an incoming bond / parameter
        self.result = result   # an outgoing bond / return value
        
        self.min_uses = 1
        if optional:
            self.min_uses = 0
            
        self.max_uses = 1
        if self.result:
            self.max_uses = 999
            # can be consumed by many other funcs
            
    def satisfied(self, rowcol):
        return self.min_uses <= rowcol <= self.max_uses

In [71]:
class Store:
    pass
atom = Store()

class Atom:
    def __init__(self, name, bonds, py):
        self.name = name
        setattr(atom, name, self)
        
        self.bonds = bonds
        for b in self.bonds:
            b.atom = self
            
        self.py = py
        
    def __str__(self):
        return '@.'+self.name
    

Atom(
    name = 'text',
    bonds = [Bond('string', result=True)],
    py = lambda : 'hello world!'
)

Atom(
    name='print',
    bonds = [Bond('string', param=True)],
    py = print
)

class Composer:
    def __init__(self, atoms):
        self.atoms = atoms
    
    def buildMaybeMasks(self):
        pass
    
        



In [ ]:
map bond type to matrix as "solution"
eval solution
tweak repeat


In [ ]:
okay build the maybe arrays
then brute force from maybe arrays

In [ ]:
build vertical and horizontal maybe masks
so row says I want one [not here, here, or here]
[0, 1, 1]

then see which direction has fewest possibilities
permute across that direction
each permutation
    check the other direction fail fast
    check the unit tests
    